### RESEARCH ON DATA FROM https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset

## IMPORTED PACKAGES

## UTIL FUNCTIONS

## MODEL TRAINING

## MODEL EVALUATION

## RESUTL

## NOTES

### AUTHOR
- LE BUI TRUNG DUNG